In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from ML.credit_risk.pipeline import make_train_pipeline
from ML.credit_risk.pipeline.data_operations import *
from ML.credit_risk.pipeline.read_data import read_data_train

from xgboost import XGBClassifier

In [3]:
data_original = read_data_train()

In [4]:
train_pipeline = make_train_pipeline()
data = train_pipeline.fit_transform(data_original.copy(deep=True))

In [16]:
x_train, x_test, y_train, y_test = split_train_data(data, ["TARGET"], 42, 0.1)

In [18]:
target_counts = y_train["TARGET"].value_counts()

In [19]:
booster = XGBClassifier(
    max_depth = 5,
    learning_rate = 1,
    scale_pos_weight = target_counts[0] / target_counts[1],
    objective = 'binary:logistic'
)

In [20]:
booster.fit(x_train, y_train)

D:\coding\Jupyter_notebooks\envML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\coding\Jupyter_notebooks\envML\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
[20:36:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=11.387386984155402,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [13]:
for col, importance in zip(x_train.columns, booster.feature_importances_):
    if importance < 0.01:
        print(f"{col}\t{importance}")

NAME_INCOME_TYPE_Businessman	0.0
NAME_INCOME_TYPE_Student	0.0043802810832858086


In [23]:
f1_score(y_test, booster.predict(x_test))

true positive: 1360
true negative: 18243
false positive: 10026
false negative: 1123


0.1961208450501118

In [24]:
cross_validate(booster, x_train, y_train, random_state=42)

(0.6292114900678327,
 array([0.62933843, 0.62510359, 0.61236054, 0.6330565 , 0.62043813,
        0.64169985, 0.62804991, 0.63231448, 0.62705365, 0.64269982]))

In [25]:
import pickle
from pathlib import Path

model_path = Path("ML/credit_risk/xgboost_roc_auc_0-63.model")
pickle.dump(booster, model_path.open("wb"))